Final Project: Part 2 

(I've entered a request for an extension.)

Problem Restatement: 

The District of Columbia Metropolitan Police Department (MPD) reported 33,890 crime incidents in 2018, nearly 700 more incidents than in the prior year. To reduce the rate of crime in the District, MPD studies the crime trends each year to determine when and where most crimes are happening. Their goal is to strategically target high crime areas with crime prevention strategis that are relevant to the crimes committed.

In conjunction there is a high request for city services, with nearly 204,318 service requests as of July 2019. This project involves developing a model to identify whether the crime index of a zipcode can predict the speed of closure for city services requests.

Target Variable: Report_To_Closure_Time

Feature set: Service Type, Agency, Priority, Ward, Zipcode, Service_Order_Time, Service_Order_Date, Crime Index

Goals and Success Metrics:

The goal is to be able to understand the extent to which the zipcode and crime index could be used to predict the closure speed for city service requestss, with the measure of success being a relatively accurate prediction of closure speed.

Risks and Limitations:

The analysis risks under estimating the correlation between service calls and crimes reported due to the small geographic area with people visiting from multiple jurisdictions for work and recreation. The project ignores the crimes that are happening in immediately adjacent jurisdictions.

The analysis also does not take into account the present disparities in provision of city services based on neighborhood, nor does it take into account areas with high homeownership versus high rentership.

Dataframe Info: 

The dataframes that will be used for this project are the City Service Request dataframe (204,319 rows x 30 columns) and the Crime Incidents 2019 (as of July 29) dataframe (18,198 rows x 25 columns). The crime index will be identified for District zipcodes, and this information will be incorporated into the City Service Request dataframe. 

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [59]:
Services = pd.read_csv('../CSV_Files/City_Service_Requests_in_2019 (csv).csv')

In [61]:
Services.head()

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,...,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS,SERVICEORDERDATE1,SERVICEORDERTIME1,RESOLUTIONDATE1,RESOLUTIONTIME1
0,-76.930018,38.904475,278741,S0031,Bulk Collection,SWMA- Solid Waste Management Admistration,DPW,1,2019-07-17T20:57:15.000Z,NaN,...,WASHINGTON,DC,20019.0,15976.0,7.0,NaN,7/17/2019,20:57,NaN,NaN
1,-77.034081,38.941341,278742,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2019-07-17T21:00:00.000Z,2019-07-20T23:03:10.000Z,...,WASHINGTON,DC,20011.0,259155.0,4.0,NaN,7/17/2019,21:00,7/20/2019,11:03:10 PM
2,-77.034326,38.915358,278743,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2019-07-17T21:58:52.000Z,2019-07-19T03:23:15.000Z,...,WASHINGTON,DC,20009.0,240017.0,2.0,NaN,7/17/2019,21:58,7/19/2019,3:23:15 AM
3,-76.942364,38.898615,278744,S0336,Out of State Parking Violation (ROSA),PEMA- Parking Enforcement Management Administr...,DPW,1,2019-07-17T21:59:10.000Z,2019-07-18T02:39:36.000Z,...,WASHINGTON,DC,20019.0,25746.0,7.0,NaN,7/17/2019,21:59,7/18/2019,2:39:36 AM
4,-76.978173,38.888846,278745,S0316,Recycling Cart Delivery,SWMA- Solid Waste Management Admistration,DPW,1,2019-07-17T21:01:44.000Z,NaN,...,WASHINGTON,DC,20003.0,55558.0,6.0,NaN,7/17/2019,21:01,NaN,NaN


In [62]:
Services.shape

(204318, 34)

In [63]:
Services.rename(columns= {'SERVICECODEDESCRIPTION':'Service_Description',
                          'SERVICETYPECODEDESCRIPTION':'Service_Type', 
                         'ORGANIZATIONACRONYM':'Agency', 
                         'ADDDATE': 'Add_Date',
                         'SERVICEDUEDATE':'Service_Due_Date',
                         'STATUS_CODE':'Status_Code',
                         'PRIORITY':'Priority',
                         'CITY':'City',
                         'STATE':'State',
                         'INSPECTIONFLAG':'Inspection_Flag',
                         'SERVICEORDERSTATUS':'Service_Order_Status',
                         'INSPECTIONDATE':'Inspection_Date', 
                         'INSPECTORNAME': 'Inspector_Name',
                         'WARD':'Ward',
                         'ZIPCODE':'Zipcode',
                         'SERVICEORDERDATE1':'Service_Order_Date',
                         'SERVICEORDERTIME1':'Service_Order_Time',
                         'RESOLUTIONDATE1':'Resolutions_Date',
                         'RESOLUTIONTIME1':'Resolutions_Time'}, 
                inplace=True)

In [64]:
Services.isnull().sum()

X                              0
Y                              0
OBJECTID                       0
SERVICECODE                    0
Service_Description            0
Service_Type                  75
Agency                        40
SERVICECALLCOUNT               0
Add_Date                       0
RESOLUTIONDATE             28700
Service_Due_Date               9
SERVICEORDERDATE               0
Inspection_Flag            19238
Inspection_Date           164471
Inspector_Name            204317
Service_Order_Status          35
Status_Code                45252
SERVICEREQUESTID               0
Priority                      42
STREETADDRESS              12385
XCOORD                         0
YCOORD                         0
LATITUDE                       0
LONGITUDE                      0
City                       12380
State                      12380
Zipcode                        5
MARADDRESSREPOSITORYID      7093
Ward                        1901
DETAILS                    88028
Service_Or

In [65]:
Services.index

RangeIndex(start=0, stop=204318, step=1)

In [66]:
Services.columns

Index(['X', 'Y', 'OBJECTID', 'SERVICECODE', 'Service_Description',
       'Service_Type', 'Agency', 'SERVICECALLCOUNT', 'Add_Date',
       'RESOLUTIONDATE', 'Service_Due_Date', 'SERVICEORDERDATE',
       'Inspection_Flag', 'Inspection_Date', 'Inspector_Name',
       'Service_Order_Status', 'Status_Code', 'SERVICEREQUESTID', 'Priority',
       'STREETADDRESS', 'XCOORD', 'YCOORD', 'LATITUDE', 'LONGITUDE', 'City',
       'State', 'Zipcode', 'MARADDRESSREPOSITORYID', 'Ward', 'DETAILS',
       'Service_Order_Date', 'Service_Order_Time', 'Resolutions_Date',
       'Resolutions_Time'],
      dtype='object')

In [67]:
Services.dtypes

X                         float64
Y                         float64
OBJECTID                    int64
SERVICECODE                object
Service_Description        object
Service_Type               object
Agency                     object
SERVICECALLCOUNT            int64
Add_Date                   object
RESOLUTIONDATE             object
Service_Due_Date           object
SERVICEORDERDATE           object
Inspection_Flag            object
Inspection_Date            object
Inspector_Name             object
Service_Order_Status       object
Status_Code                object
SERVICEREQUESTID           object
Priority                   object
STREETADDRESS              object
XCOORD                    float64
YCOORD                    float64
LATITUDE                  float64
LONGITUDE                 float64
City                       object
State                      object
Zipcode                   float64
MARADDRESSREPOSITORYID    float64
Ward                      float64
DETAILS       

In [72]:
City_Services = Services.drop(['X', 'Y', 'OBJECTID', 'SERVICECODE',  'SERVICECALLCOUNT', 
                               'SERVICEREQUESTID', 'RESOLUTIONDATE', 'Priority', 'STREETADDRESS', 'XCOORD', 
                               'YCOORD', 'LATITUDE', 'LONGITUDE',  
                               'MARADDRESSREPOSITORYID', 'DETAILS'], axis=1)

In [74]:
City_Services.head()

,Service_Description,Service_Type,Agency,Add_Date,Service_Due_Date,SERVICEORDERDATE,Inspection_Flag,Inspection_Date,Inspector_Name,Service_Order_Status,Status_Code,City,State,Zipcode,Ward,Service_Order_Date,Service_Order_Time,Resolutions_Date,Resolutions_Time
0,Bulk Collection,SWMA- Solid Waste Management Admistration,DPW,2019-07-17T20:57:15.000Z,2019-08-08T23:59:00.000Z,2019-07-17T20:57:15.000Z,0,2019-07-17T20:58:03.000Z,NaN,Open,NaN,WASHINGTON,DC,20019.0,7.0,7/17/2019,20:57,NaN,NaN
1,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,2019-07-17T21:00:00.000Z,2019-07-26T21:00:00.000Z,2019-07-17T21:00:00.000Z,0,2019-07-20T23:03:09.000Z,NaN,Closed,NaN,WASHINGTON,DC,20011.0,4.0,7/17/2019,21:00,7/20/2019,11:03:10 PM
2,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,2019-07-17T21:58:52.000Z,2019-07-28T21:58:52.000Z,2019-07-17T21:58:52.000Z,0,2019-07-19T03:23:14.000Z,NaN,Closed,NaN,WASHINGTON,DC,20009.0,2.0,7/17/2019,21:58,7/19/2019,3:23:15 AM
3,Out of State Parking Violation (ROSA),PEMA- Parking Enforcement Management Administr...,DPW,2019-07-17T21:59:10.000Z,2019-08-28T21:59:09.000Z,2019-07-17T21:59:10.000Z,NaN,NaN,NaN,Closed,NaN,WASHINGTON,DC,20019.0,7.0,7/17/2019,21:59,7/18/2019,2:39:36 AM
4,Recycling Cart Delivery,SWMA- Solid Waste Management Admistration,DPW,2019-07-17T21:01:44.000Z,2019-08-14T21:01:44.000Z,2019-07-17T21:01:44.000Z,0,2019-07-18T08:17:00.000Z,NaN,Open,NaN,WASHINGTON,DC,20003.0,6.0,7/17/2019,21:01,NaN,NaN


In [76]:
City_Services['Report_To_Closure_Time'] = City_Services.Resolutions_Date - City_Services.Service_Order_Date

TypeError: unsupported operand type(s) for -: 'float' and 'str'

In [ ]:
#Need to resolve date and time responses issue

feature_cols = ['Zipcode', 'Service Type', 'Agency', 'Priority', 'Ward', 'Zip_Code', 'Resolutions_Time', 'Resolution_Date']
target_col = 'Report_To_Closure_Time'

X=City_Services.loc[:, feature_cols]
y=City_Services.loc[:, target_col]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
y_test.value_counts(normalize=True)

In [ ]:
lr.predict_proba(X_test)[:,1]

In [ ]:
lr.predict(X_test)

In [ ]:
ax=City_Services.plot(kind='scatter', x='Zip_Code', y= 'Report_To_Closure_Time', alpha=.2)
x_grid = np.linspace(start=City_Services.loc[:, 'Zip_Code'].min(),
                     stop=City_Services.loc[:, 'Zip_Code'].max(),)

y_pred_prob = lr.predict_proba(x_grid.reshape(50,1))[:,1]

ax.plot(x_grid, y_pred_prob, c='g')

y_pred = lr.predict(x_grid.reshape(-1,1))
ax.plot(x_grid, y_pred, c='r')